In [1]:
import json
import pandas as pd

# Specify the path to your JSON file
file_path = "vert-plan-uta2026.json"

# Load and parse the JSON file
try:
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except FileNotFoundError:
    print("File not found. Please ensure the file path is correct.")

In [2]:
def process_workout_step(step, prefix):
    result = ''
    if step.get('type','') == 'repeat':
        result += f'{prefix}{process_workout_repeats(step, prefix)}'
    elif step.get('type','') == 'step':
        result += f'{prefix}[{step['variable']}] {step['action']} for {step['value']}'
        if step['targetType'] != 'No Target':
            result += f' at {step['targetValue']} {step['targetType']}'
        result += '<br/>'

    return result

def process_workout_repeats(step, prefix):
    result = f'{prefix}Repeat {step.get('times', -1)} times:<br/> <ul>'
    for substep in step.get('steps',[]):
        result += process_workout_step(substep, prefix+'<li>') + '</li>'
    return result + '</ul>'

def process_workout(steps):
    result = ''
    for step in steps:
        result += process_workout_step(step, '')
    return result


In [3]:
def map_training(training):
    keys_to_retain = ["trainingPeriod", "workoutType", "surface", "difficultyLevel", "estimatedTime", "parametrizedWorkout", "description", "id", "title", "trainingTime", "plannedDate"]
    result = {key: training.get(key, None) for key in keys_to_retain }
    result["description"] = result.get("description", {"en":""})["en"]
    result["title"] = result.get("title", {"en":""})["en"]
    result["estimatedTime"] = result.get("estimatedTime", ['0','0','0'])[1]
    result["parametrizedWorkout"] = result.get("parametrizedWorkout", [[],[],[]])[1]
    result["renderedWorkout"] = process_workout(result["parametrizedWorkout"])
    return result



'''
[
[{'variable': 'Time', 'targetValue': '', 'action': 'Warm up', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739524841, 'type': 'step', 'value': '00h:20m:00s'}, {'times': 4, 'id': 1666739548134, 'type': 'repeat', 'steps': [{'variable': 'Time', 'targetValue': '8', 'action': 'Run Uphill', 'isDeleteOpen': False, 'targetType': 'RPE', 'id': 1666739554725, 'type': 'step', 'value': '00h:03m:00s', 'selected': False, 'chosen': False}, {'variable': 'Time', 'targetValue': '', 'action': 'Recover', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739575154, 'type': 'step', 'value': '00h:03m:00s', 'selected': False, 'chosen': False}]}, {'variable': 'Time', 'targetValue': '', 'action': 'Cool down', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739617873, 'type': 'step', 'value': '00h:15m:00s'}],

[{'variable': 'Time', 'targetValue': '', 'action': 'Warm up', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739524841, 'type': 'step', 'value': '00h:20m:00s'}, {'times': 5, 'id': 1666739548134, 'type': 'repeat', 'steps': [{'variable': 'Time', 'targetValue': '8', 'action': 'Run Uphill', 'isDeleteOpen': False, 'targetType': 'RPE', 'id': 1666739554725, 'type': 'step', 'value': '00h:03m:00s', 'selected': False, 'chosen': False}, {'variable': 'Time', 'targetValue': '', 'action': 'Recover', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739575154, 'type': 'step', 'value': '00h:03m:00s', 'selected': False, 'chosen': False}]}, {'variable': 'Time', 'targetValue': '', 'action': 'Cool down', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739617873, 'type': 'step', 'value': '00h:25m:00s'}],

[{'variable': 'Time', 'targetValue': '', 'action': 'Warm up', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739524841, 'type': 'step', 'value': '00h:20m:00s'}, {'times': 6, 'id': 1666739548134, 'type': 'repeat', 'steps': [{'variable': 'Time', 'targetValue': '8', 'action': 'Run Uphill', 'isDeleteOpen': False, 'targetType': 'RPE', 'id': 1666739554725, 'type': 'step', 'value': '00h:03m:00s', 'selected': False, 'chosen': False}, {'variable': 'Time', 'targetValue': '', 'action': 'Recover', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739575154, 'type': 'step', 'value': '00h:03m:00s', 'selected': False, 'chosen': False}]}, {'variable': 'Time', 'targetValue': '', 'action': 'Cool down', 'isDeleteOpen': False, 'targetType': 'No Target', 'id': 1666739617873, 'type': 'step', 'value': '00h:20m:00s'}]]
'''

training_days_raw = pd.DataFrame([map_training(session[0]) for session in data['days'] if session[0].get("isDone",False) == False])
training_days = training_days_raw.drop(columns=['parametrizedWorkout', 'id'])


In [4]:
training_days = training_days[[    'plannedDate', 'workoutType', 'title', 'description', 'renderedWorkout', 'difficultyLevel', 'trainingTime', 'trainingPeriod', 'surface', 'estimatedTime' ]]


In [5]:
training_days.rename(columns={'trainingPeriod': 'Period',
                              'workoutType': 'Type',
                              'surface': 'Surface',
                              'difficultyLevel': 'Difficulty',
                              'description': 'Description',
                              'title': 'Title',
                              'trainingTime': 'Time',
                              'plannedDate': 'Date',
                              'renderedWorkout': 'Workout Detail',
                              }, inplace=True)
training_days


,Date,Type,Title,Description,Workout Detail,Difficulty,Time,Period,Surface,estimatedTime
0,24/09/2025,EasyRun,03 / BB Medium* / Wednesday / Easy run + strides,<p>&nbsp;</p><h3>Today's training: easy run wi...,[Time] Warm up for 00h:30m:00s<br/>Repeat 4 ti...,None,AllDay,Specific,Flat,45
1,25/09/2025,Corework,03 / BB Medium* / Thursday / Easy run + Condit...,<p>&nbsp;</p><h3>Today's training: easy run + ...,[Time] Run for 00h:30m:00s at 4 RPE<br/>[Time]...,None,AllDay,Specific,Flat,45
2,26/09/2025,Recovery,Rest*1,<p>&nbsp;</p><h3>Today's training: rest.&nbsp;...,,3,AllDay,Recovering,Flat,0
3,27/09/2025,LongRun,03 / BB Medium* / Saturday / Long run,<p>&nbsp;</p><h3>Today's training: easy long r...,[Time] Warm up for 00h:15m:00s<br/>[Time] Run ...,None,AllDay,Specific,Flat,75
4,28/09/2025,EasyRun,03 / BB Medium* / Sunday / Easy run,<p>&nbsp;</p><h3>Today’s training: easy run.</...,[Time] Run for 00h:45m:00s at 4 RPE<br/>,None,AllDay,Specific,Flat,45
...,...,...,...,...,...,...,...,...,...,...
231,13/05/2026,EasyRun,12 / Climb+Endurance 50k* / Wednesday / Easy run,<p>&nbsp;</p><h3>Today's training: easy run.&n...,[Time] Run for 00h:45m:00s at 4 RPE<br/>,1,AllDay,Basic,Flat,45
232,14/05/2026,Speedwork,12 / Climb+Endurance 50k* / Thursday / Easy ru...,<p>&nbsp;</p><h3>Today's training: easy run wi...,[Time] Warm up for 00h:15m:00s<br/>Repeat 5 ti...,1,AllDay,Competition,Flat,30
233,15/05/2026,Recovery,Rest*1,<p>&nbsp;</p><h3>Today's training: rest.&nbsp;...,,3,AllDay,Recovering,Flat,0
234,16/05/2026,RaceDay,12 / Climb+Endurance 50k* / Saturday / Final race,<p>&nbsp;</p><h3>RACE DAY</h3><p>Time to put o...,,3,AllDay,Competition,Mountains,0


In [6]:
# Assign class to the rows so that we can scroll to today
classes = pd.DataFrame('', index=training_days.index, columns=training_days.columns)
classes.loc[:, 'Date'] = training_days['Date']

In [ ]:
from IPython.display import display, HTML

styled_table = training_days.style.format({
    "Description": lambda x: x,
    "title": lambda x: x,
    "Workout Detail":lambda x: x,
    }).set_properties(**{'font-size': '11pt', 'font-family': 'Arial', 'border': '1px solid #000000'}).hide(axis="index")  # Apply HTML format to the "description" column

styled_table.set_td_classes(classes)

code = styled_table.to_html(escape=False, classes='table')


# Wrap the table HTML in a full HTML document that includes JavaScript to scroll to the element with class "today"
html_content = f"""
<html>
<head>
  <meta charset="UTF-8">
  <title>UTA50 2025 Training Plan</title>
  <script>
      function scrollToToday() {{
        const today = new Date();
        const dd = String(today.getDate()).padStart(2, '0');
        const mm = String(today.getMonth() + 1).padStart(2, '0'); // Months are zero-indexed
        const yyyy = today.getFullYear();

        const formattedDate = `${{dd}}/${{mm}}/${{yyyy}}`;
      // Find the first element with the "today" class
      var el = document.getElementsByClassName(formattedDate)[0];
      if (el) {{
        el.scrollIntoView({{behavior: 'smooth', block: 'center'}});
      }}
    }};
    window.onload = scrollToToday;
  </script>
</head>
<body>
   <button onclick="scrollToToday()">Scroll to Today's Date</button>
  {code}
</body>
</html>
"""

with open('plan.html', 'w') as f:
    f.write(html_content)
display(html_content)


'\n<html>\n<head>\n  <meta charset="UTF-8">\n  <title>UTA50 2025 Training Plan</title>\n  <script>\n      function scrollToToday() {\n        const today = new Date();\n        const dd = String(today.getDate()).padStart(2, \'0\');\n        const mm = String(today.getMonth() + 1).padStart(2, \'0\'); // Months are zero-indexed\n        const yyyy = today.getFullYear();\n\n        const formattedDate = `${dd}/${mm}/${yyyy}`;\n      // Find the first element with the "today" class\n      var el = document.getElementsByClassName(formattedDate)[0];\n      if (el) {\n        el.scrollIntoView({behavior: \'smooth\', block: \'center\'});\n      }\n    };\n    window.onload = scrollToToday;\n  </script>\n</head>\n<body>\n   <button onclick="scrollToToday()">Scroll to Today\'s Date</button>\n  <style type="text/css">\n#T_6dddc_row0_col0, #T_6dddc_row0_col1, #T_6dddc_row0_col2, #T_6dddc_row0_col3, #T_6dddc_row0_col4, #T_6dddc_row0_col5, #T_6dddc_row0_col6, #T_6dddc_row0_col7, #T_6dddc_row0_col8, 